Import everything you need — including the model class (here it's GCNConv).

In [2]:
import torch
from torch_geometric.loader import DataLoader
import torch.nn.functional as F
from torch.nn import Linear, Sequential, BatchNorm1d, ReLU, Dropout # For the multi-perceptron layer
from torch_geometric.nn import GINConv  # Load graph isomorphism network
from torch_geometric.nn import global_mean_pool, global_add_pool
from torch_geometric.data import Data
from sklearn.metrics import accuracy_score, mean_squared_error

# 🔁 Load datasets
task = "regression"

train_data = torch.load(f"../4_train_test_split/random_split/{task}/{task}_train.pt")
val_data = torch.load(f"../4_train_test_split/random_split/{task}/{task}_val.pt")
test_data = torch.load(f"../4_train_test_split/random_split/{task}/{task}_test.pt")

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_data, batch_size=32)

# 🧱 Define Model (GIN)
class GIN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels): # where "dim_h" is dimensionality of layer; number of features or neurons that layer processes
        super(GIN, self).__init__()
        self.conv1 = GINConv( # design of the multilayer perceptron layer for the GINConv layer based on the paper (https://arxiv.org/pdf/1905.12265)
            Sequential(Linear(in_channels, hidden_channels),
                       BatchNorm1d(hidden_channels), ReLU(),
                       Linear(hidden_channels, hidden_channels), ReLU()))
        self.conv2 = GINConv(
            Sequential(Linear(hidden_channels, hidden_channels), BatchNorm1d(hidden_channels), ReLU(),
                       Linear(hidden_channels, hidden_channels), ReLU()))
        self.conv3 = GINConv(
            Sequential(Linear(hidden_channels, hidden_channels), BatchNorm1d(hidden_channels), ReLU(),
                       Linear(hidden_channels, hidden_channels), ReLU()))
        self.lin1 = Linear(hidden_channels*3, hidden_channels*3)
        self.lin2 = Linear(hidden_channels*3, out_channels)

    def forward(self, data):
        # Node embeddings
        h1 = self.conv1(data.x, data.edge_index) 
        h2 = self.conv2(h1, data.edge_index)
        h3 = self.conv3(h2, data.edge_index)

        # Graph level readout
        h1 = global_add_pool(h1, data.batch)
        h2 = global_add_pool(h2, data.batch)
        h3 = global_add_pool(h3, data.batch)

        # Concatenate graph embeddings
        h = torch.cat((h1, h2, h3), dim=1)

        # Classifier
        h = self.lin1(h)
        h = h.relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.lin2(h)

        return h

# 🧮 Loss and evaluation setup
if task == "classification":
    num_classes = len(set([int(data.y.item()) for data in train_data]))
    model = GIN(in_channels=train_data[0].x.size(1), hidden_channels=64, out_channels=num_classes)
    criterion = torch.nn.CrossEntropyLoss()
else:
    model = GIN(in_channels=train_data[0].x.size(1), hidden_channels=64, out_channels=1)
    criterion = torch.nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 🏋️ Training loop
def train():
    model.train()
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch)
        loss = criterion(out.squeeze(), batch.y if task == "regression" else batch.y.long())
        loss.backward()
        optimizer.step()

# 📈 Evaluation
def evaluate(loader):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            out = model(batch)
            preds.append(out.squeeze().cpu())
            labels.append(batch.y.cpu())
    preds = torch.cat(preds)
    labels = torch.cat(labels)

    if task == "classification":
        pred_classes = preds.argmax(dim=1)
        return accuracy_score(labels, pred_classes)
    else:
        return mean_squared_error(labels, preds)

# 🚀 Run training
for epoch in range(1, 101):
    train()
    metric = evaluate(val_loader)
    print(f"Epoch {epoch:03d} - {'Accuracy' if task == 'classification' else 'MSE'}: {metric:.4f}")

# ✅ Final test evaluation
test_metric = evaluate(test_loader)
print(f"\n🧪 Test {'Accuracy' if task == 'classification' else 'MSE'}: {test_metric:.4f}")


Epoch 001 - MSE: 1.9008
Epoch 002 - MSE: 1.6460
Epoch 003 - MSE: 0.8651
Epoch 004 - MSE: 0.7806
Epoch 005 - MSE: 1.0175
Epoch 006 - MSE: 0.6677
Epoch 007 - MSE: 0.7444
Epoch 008 - MSE: 0.7887
Epoch 009 - MSE: 0.8355
Epoch 010 - MSE: 0.7227
Epoch 011 - MSE: 0.7612
Epoch 012 - MSE: 0.6709
Epoch 013 - MSE: 0.7838
Epoch 014 - MSE: 0.8988
Epoch 015 - MSE: 0.8353
Epoch 016 - MSE: 0.8677
Epoch 017 - MSE: 0.8791
Epoch 018 - MSE: 1.0846
Epoch 019 - MSE: 0.8720
Epoch 020 - MSE: 0.6762
Epoch 021 - MSE: 1.3621
Epoch 022 - MSE: 0.7743
Epoch 023 - MSE: 0.6211
Epoch 024 - MSE: 0.7758
Epoch 025 - MSE: 0.5443
Epoch 026 - MSE: 0.6965
Epoch 027 - MSE: 0.7294
Epoch 028 - MSE: 0.7967
Epoch 029 - MSE: 0.8439
Epoch 030 - MSE: 0.7600
Epoch 031 - MSE: 1.1824
Epoch 032 - MSE: 1.1284
Epoch 033 - MSE: 0.6688
Epoch 034 - MSE: 0.7032
Epoch 035 - MSE: 0.5703
Epoch 036 - MSE: 0.6966
Epoch 037 - MSE: 0.8152
Epoch 038 - MSE: 1.0578
Epoch 039 - MSE: 0.8665
Epoch 040 - MSE: 0.6527
Epoch 041 - MSE: 0.7156
Epoch 042 - MSE:

load the graph datasets  (the random split .pt files ) and specify what task you are doing

In [1]:
task = "regression"

train_data = torch.load(f"../4_train_test_split/random_split/{task}/{task}_train.pt")
val_data = torch.load(f"../4_train_test_split/random_split/{task}/{task}_val.pt")
test_data = torch.load(f"../4_train_test_split/random_split/{task}/{task}_test.pt")

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_data, batch_size=32)

NameError: name 'torch' is not defined

Define the model you want to use (GCN or MPNN or GraphSAGE, etc.)

In [24]:
# 🧱 Define Model (GIN)
class GIN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels): # where "dim_h" is dimensionality of layer; number of features or neurons that layer processes
        super(GIN, self).__init__()
        self.conv1 = GINConv( # design of the multilayer perceptron layer for the GINConv layer based on the paper (https://arxiv.org/pdf/1905.12265)
            Sequential(Linear(in_channels, hidden_channels),
                       BatchNorm1d(hidden_channels), ReLU(),
                       Linear(hidden_channels, hidden_channels), ReLU()))
        self.conv2 = GINConv(
            Sequential(Linear(hidden_channels, hidden_channels), BatchNorm1d(hidden_channels), ReLU(),
                       Linear(hidden_channels, hidden_channels), ReLU()))
        self.conv3 = GINConv(
            Sequential(Linear(hidden_channels, hidden_channels), BatchNorm1d(hidden_channels), ReLU(),
                       Linear(hidden_channels, hidden_channels), ReLU()))
        self.lin1 = Linear(hidden_channels*3, hidden_channels*3)
        self.lin2 = Linear(hidden_channels*3, out_channels)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        # Node embeddings
        h1 = self.conv1(x, edge_index) 
        h2 = self.conv2(h1, edge_index)
        h3 = self.conv3(h2, edge_index)

        # Graph level readout
        h1 = global_add_pool(h1, batch) 
        h2 = global_add_pool(h2, batch)
        h3 = global_add_pool(h3, batch)

        # Concatenate graph embeddings
        h = torch.cat((h1, h2, h3), dim=1)

        # Classifier
        h = self.lin1(h)
        h = h.relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.lin2(h)

        return h

setup loss and evaluation 

In [25]:
if task == "classification":
    num_classes = len(set([int(data.y.item()) for data in train_data]))
    model = GIN(in_channels=train_data[0].x.size(1), hidden_channels=64, out_channels=num_classes)
    criterion = torch.nn.CrossEntropyLoss()
else:
    model = GIN(in_channels=train_data[0].x.size(1), hidden_channels=64, out_channels=1)
    criterion = torch.nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 🏋️ Training loop
def train():
    model.train()
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch)
        loss = criterion(out.squeeze(), batch.y if task == "regression" else batch.y.long())
        loss.backward()
        optimizer.step()

# 📈 Evaluation
def evaluate(loader):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            out = model(batch)
            preds.append(out.squeeze().cpu())
            labels.append(batch.y.cpu())
    preds = torch.cat(preds)
    labels = torch.cat(labels)

    if task == "classification":
        pred_classes = preds.argmax(dim=1)
        return accuracy_score(labels, pred_classes)
    else:
        return mean_squared_error(labels, preds)


🚀 Model training

In [26]:
for epoch in range(1, 101):
    train()
    metric = evaluate(val_loader)
    print(f"Epoch {epoch:03d} - {'Accuracy' if task == 'classification' else 'MSE'}: {metric:.4f}")

# ✅ Final test evaluation
test_metric = evaluate(test_loader)
print(f"\n🧪 Test {'Accuracy' if task == 'classification' else 'MSE'}: {test_metric:.4f}")

Epoch 001 - Accuracy: 0.5192
Epoch 002 - Accuracy: 0.6731
Epoch 003 - Accuracy: 0.7115
Epoch 004 - Accuracy: 0.7115
Epoch 005 - Accuracy: 0.6923
Epoch 006 - Accuracy: 0.6923
Epoch 007 - Accuracy: 0.7115
Epoch 008 - Accuracy: 0.6923
Epoch 009 - Accuracy: 0.6731
Epoch 010 - Accuracy: 0.6731
Epoch 011 - Accuracy: 0.7500
Epoch 012 - Accuracy: 0.7500
Epoch 013 - Accuracy: 0.7500
Epoch 014 - Accuracy: 0.7115
Epoch 015 - Accuracy: 0.6731
Epoch 016 - Accuracy: 0.6923
Epoch 017 - Accuracy: 0.6923
Epoch 018 - Accuracy: 0.6538
Epoch 019 - Accuracy: 0.6923
Epoch 020 - Accuracy: 0.6923
Epoch 021 - Accuracy: 0.5577
Epoch 022 - Accuracy: 0.7115
Epoch 023 - Accuracy: 0.6923
Epoch 024 - Accuracy: 0.6923
Epoch 025 - Accuracy: 0.6731
Epoch 026 - Accuracy: 0.6154
Epoch 027 - Accuracy: 0.7500
Epoch 028 - Accuracy: 0.6538
Epoch 029 - Accuracy: 0.6538
Epoch 030 - Accuracy: 0.7115
Epoch 031 - Accuracy: 0.6923
Epoch 032 - Accuracy: 0.5769
Epoch 033 - Accuracy: 0.6731
Epoch 034 - Accuracy: 0.6923
Epoch 035 - Ac